In [1]:
import json
from time import sleep
from threading import Thread
from os.path import join, exists
from traceback import print_exc
from random import random
from datetime import datetime, timedelta

from api.dwx_client import dwx_client

In [4]:
mt_dir = 'C:/Users/nikki/AppData/Roaming/MetaQuotes/Terminal/9313AE53F35B199F6134FA1F544EA725/MQL4/Files/'

test = dwx_client(mt_dir,)

ERROR: metatrader_dir_path does not exist!


NameError: name 'exit' is not defined

In [12]:
dwx_client.mro()

[api.dwx_client.dwx_client, object]

In [5]:
class test():
    
    def __init__(self, mt_dir,
                 sleep_delay=0.005,
                 max_retry_command_seconds=10,
                 verbose=True
                ):
        
        self.dwx = dwx_client(self, mt_dir, sleep_delay,
                              max_retry_command_seconds, verbose)
        sleep(1)
        
        self.dwx.start()
        
        print("Account info: ", self.dwx.account_info)
        
        self.dwx.get_historic_data('EURUSD', 'D1')
        
    def on_historic_data(self, symbol, time_frame, data):
        
        # you can also access the historic data via self.dwx.historic_data. 
        print('historic_data:', symbol, time_frame, f'{len(data)} bars')



In [10]:
help(test)

Help on class test in module __main__:

class test(builtins.object)
 |  test(mt_dir, sleep_delay=0.005, max_retry_command_seconds=10, verbose=True)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, mt_dir, sleep_delay=0.005, max_retry_command_seconds=10, verbose=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [2]:
"""

Example dwxconnect client in python


This example client will subscribe to tick data and bar data. It will also request historic data. 
if open_test_trades=True, it will also open trades. Please only run this on a demo account. 

"""

class tick_processor():

    def __init__(self, MT4_directory_path, 
                 sleep_delay=0.005,             # 5 ms for time.sleep()
                 max_retry_command_seconds=10,  # retry to send the commend for 10 seconds if not successful. 
                 verbose=True
                 ):

        # if true, it will randomly try to open and close orders every few seconds. 
        self.open_test_trades = False

        self.last_open_time = datetime.utcnow()
        self.last_modification_time = datetime.utcnow()

        self.dwx = dwx_client(self, MT4_directory_path, sleep_delay, 
                              max_retry_command_seconds, verbose=verbose)
        sleep(1)

        self.dwx.start()
        
        # account information is stored in self.dwx.account_info.
        print("Account info:", self.dwx.account_info)

        # subscribe to tick data:
        self.dwx.subscribe_symbols(['EURUSD', 'GBPUSD'])

        # subscribe to bar data:
        self.dwx.subscribe_symbols_bar_data([['EURUSD', 'M15'], ['GBPJPY', 'M5'], ['AUDCAD', 'M1']])

        # request historic data:
        end = datetime.utcnow()
        start = end - timedelta(days=30)  # last 30 days
        self.dwx.get_historic_data('EURUSD', 'D1', start.timestamp(), end.timestamp())


    def on_tick(self, symbol, bid, ask):

        now = datetime.utcnow()

        print('on_tick:', now, symbol, bid, ask)

        # to test trading. 
        # this will randomly tryp to open and close orders every few seconds. 
        if self.open_test_trades:
            if now > self.last_open_time + timedelta(seconds=3):

                self.last_open_time = now
                
                order_type = 'buy'
                price = ask
                if random() > 0.5:
                    order_type = 'sell'
                    price = bid

                self.dwx.open_order(symbol=symbol, order_type=order_type, 
                                    price=price, lots=0.5)

            if now > self.last_modification_time + timedelta(seconds=10):

                self.last_modification_time = now

                for ticket in self.dwx.open_orders.keys():
                    self.dwx.close_order(ticket, lots=0.1)

            if len(self.dwx.open_orders) >= 10:
                self.dwx.close_all_orders()
                # self.dwx.close_orders_by_symbol('GBPUSD')
                # self.dwx.close_orders_by_magic(0)


    def on_bar_data(self, symbol, time_frame, time, open_price, high, low, close_price, tick_volume):
        
        print('on_bar_data:', symbol, time_frame, datetime.utcnow(), time, open_price, high, low, close_price)

    
    def on_historic_data(self, symbol, time_frame, data):
        
        # you can also access the historic data via self.dwx.historic_data. 
        print('historic_data:', symbol, time_frame, f'{len(data)} bars')


    def on_historic_trades(self):

        print(f'historic_trades: {len(self.dwx.historic_trades)}')
    

    def on_message(self, message):

        if message['type'] == 'ERROR':
            print(message['type'], '|', message['error_type'], '|', message['description'])
        elif message['type'] == 'INFO':
            print(message['type'], '|', message['message'])


    # triggers when an order is added or removed, not when only modified. 
    def on_order_event(self):
        
        print(f'on_order_event. open_orders: {len(self.dwx.open_orders)} open orders')



MT4_files_dir = 'C:/Users/nikki/AppData/Roaming/MetaQuotes/Terminal/9313AE53F35B199F6134FA1F544EA725/MQL4/Files/'

processor = tick_processor(MT4_files_dir)

while processor.dwx.ACTIVE:
    sleep(1)


Account info: {}
INFO | Successfully subscribed to: EURUSD, GBPUSD
INFO | on_bar_data:Successfully subscribed to bar data: EURUSD,M15,GBPJPY,M5,AUDCAD,M1
 EURUSD M15 2021-07-10 07:10:45.875577 2021.07.09 23:30 1.18789 1.18793 1.18779 1.18786
on_bar_data: GBPJPY M5 2021-07-10 07:10:45.878578 2021.07.09 23:50 153.062 153.062 153.005 153.036
on_bar_data: AUDCAD M1 2021-07-10 07:10:45.878578 2021.07.09 23:55 0.93247 0.93265 0.9324 0.9325
historic_data: EURUSD D1 21 bars
INFO | Successfully read historic data for EURUSD_D1.
on_tick: 2021-07-10 07:10:45.893575 EURUSD 1.18773 1.18777
on_tick: 2021-07-10 07:10:45.896575 GBPUSD 1.38984 1.39009


KeyboardInterrupt: 

In [5]:
composite = yf.download('^JKSE', interval='1mo')

[*********************100%***********************]  1 of 1 completed


In [22]:
composite['pct_change'] = composite['Adj Close'].pct_change()
composite

,Open,High,Low,Close,Adj Close,Volume,pct_change
Date,,,,,,,
1990-05-01,639.010010,647.401001,636.153992,636.400024,636.400024,0,NaN
1990-06-01,635.020020,637.080017,621.789978,624.320007,624.320007,0,-0.018982
1990-07-01,629.440002,631.169983,614.406006,614.406006,614.406006,0,-0.015880
1990-08-01,611.981018,613.494019,546.830017,571.020020,571.020020,0,-0.070615
1990-09-01,567.406982,567.406982,454.511993,468.509003,468.509003,0,-0.179523
...,...,...,...,...,...,...,...
2021-03-01,6281.856934,6394.452148,5892.645020,5985.521973,5985.521973,3451452000,-0.041058
2021-04-01,5988.022949,6115.620117,5883.523926,5995.616211,5995.616211,2813253600,0.001686
2021-05-01,5999.661133,6005.087891,5742.038086,5947.462891,5947.462891,2453433400,-0.008031


In [21]:
composite.drop(composite.index[-1], inplace=True)

In [20]:
composite

,Open,High,Low,Close,Adj Close,Volume,pct_change
Date,,,,,,,
1990-05-01,639.010010,647.401001,636.153992,636.400024,636.400024,0,NaN
1990-06-01,635.020020,637.080017,621.789978,624.320007,624.320007,0,-0.018982
1990-07-01,629.440002,631.169983,614.406006,614.406006,614.406006,0,-0.015880
1990-08-01,611.981018,613.494019,546.830017,571.020020,571.020020,0,-0.070615
1990-09-01,567.406982,567.406982,454.511993,468.509003,468.509003,0,-0.179523
...,...,...,...,...,...,...,...
2021-04-01,5988.022949,6115.620117,5883.523926,5995.616211,5995.616211,2813253600,0.001686
2021-05-01,5999.661133,6005.087891,5742.038086,5947.462891,5947.462891,2453433400,-0.008031
2021-06-01,6002.480957,6134.881836,5884.917969,5985.488770,5985.488770,3980614000,0.006394


In [18]:
composite.index[-1]

Timestamp('2021-07-08 00:00:00')

In [23]:
composite.groupby(composite.index.strftime('%B'))['pct_change'].mean()

Date
April        0.020559
August      -0.032455
December     0.043907
February     0.006216
January      0.021912
July         0.011969
June         0.012878
March        0.002840
May          0.018620
November     0.012203
October      0.001425
September   -0.014563
Name: pct_change, dtype: float64